# Preparation

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/HR_comma_sep (2).csv')

In [ ]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [ ]:
df.sample(5)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
3397,0.77,0.49,4,184,3,0,0,0,technical,low
3196,0.66,0.59,3,204,3,0,0,0,product_mng,high
8311,0.70,0.60,3,187,2,0,0,0,product_mng,high
2551,0.72,0.62,4,256,3,0,0,0,hr,low
13521,0.77,0.74,5,243,2,0,0,0,technical,low


In [ ]:
df.isnull().sum()

satisfaction_level       0
last_evaluation          0
number_project           0
average_montly_hours     0
time_spend_company       0
Work_accident            0
left                     0
promotion_last_5years    0
sales                    0
salary                   0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   average_montly_hours   14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  int64  
 6   left                   14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   sales                  14999 non-null  object 
 9   salary                 14999 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


# Preprocessing

In [ ]:
df = pd.get_dummies(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   average_montly_hours   14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  int64  
 6   left                   14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   sales_IT               14999 non-null  uint8  
 9   sales_RandD            14999 non-null  uint8  
 10  sales_accounting       14999 non-null  uint8  
 11  sales_hr               14999 non-null  uint8  
 12  sales_management       14999 non-null  uint8  
 13  sales_marketing        14999 non-null  uint8  
 14  sales_product_mng      14999 non-null  uint8  
 15  sa

In [ ]:
X = df.drop(['left'],1)
y = df['left']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [ ]:
X_train.shape

(10499, 20)

In [ ]:
X_test.shape

(4500, 20)

# K-Nearest Neighbor

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [ ]:
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
y_pred = knn.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9306666666666666

In [ ]:
# baseline
y_test.value_counts(normalize=True)

0    0.762
1    0.238
Name: left, dtype: float64

# Tuning Hyperparameter

In [ ]:
params = {'n_neighbors':[1,2,3,4,5,6,7,8,9,10]} # hati-hati pemilihan hyperparameter jangan terlalu banyak kombinasinya

In [ ]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=knn,
                    param_grid=params,
                    scoring='accuracy',
                    n_jobs=10,
                    cv=3)

In [ ]:
grid.fit(X_train,y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=10,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [ ]:
grid.best_params_

{'n_neighbors': 6}

In [ ]:
y_pred = grid.predict(X_test)

In [ ]:
accuracy_score(y_test,y_pred)

0.9375555555555556

# Tuning Hyperparameter Manual

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [ ]:
y_pred = knn.predict(X_test)
accuracy_score(y_test,y_pred)

0.9366666666666666

In [ ]:
iterasi = range(1,31)
k = []
scores = []
for i in iterasi:

  knn = KNeighborsClassifier(n_neighbors=i)
  knn.fit(X_train,y_train)
  y_pred = knn.predict(X_test)
  accuracy = accuracy_score(y_test,y_pred)

  k.append(i)
  scores.append(accuracy)
result = {'k':k,
          'acc':scores}

In [ ]:
result_df = pd.DataFrame(result)

In [ ]:
result_df.sort_values('acc',ascending=False)

,k,acc
0,1,0.950667
1,2,0.944889
3,4,0.940222
7,8,0.940000
9,10,0.938000
5,6,0.937556
13,14,0.937111
2,3,0.936667
11,12,0.936444
15,16,0.936222


# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()

In [ ]:
dtc.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
y_pred = dtc.predict(X_test)
accuracy_score(y_test,y_pred)

0.9777777777777777

# Tuning Hyperparameter Decision Tree

In [60]:
dtc = DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_split=20, min_samples_leaf=10)

In [61]:
dtc.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=3, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=20,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [62]:
y_pred = dtc.predict(X_test)
accuracy_score(y_test,y_pred)

0.9422222222222222

In [68]:
params = {'max_depth':[3,5,7,9],
          'min_samples_split':[20,50,0],
          'min_samples_leaf':[10,30,0],
          'criterion':['entropy']}

In [69]:
4 * 3 * 3 * 1 * 3

108

In [70]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(dtc, param_grid=params, scoring='accuracy', n_jobs=10, cv=3)
grid.fit(X_train,y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='entropy', max_depth=3,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=10,
                                              min_samples_split=20,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=10,
             param_grid={'criterion': ['entropy'], 'max_depth': [3, 5, 7, 9],


In [71]:
grid.best_params_

{'criterion': 'entropy',
 'max_depth': 7,
 'min_samples_leaf': 10,
 'min_samples_split': 50}

In [72]:
y_pred = grid.predict(X_test)
accuracy_score(y_test, y_pred)

0.9742222222222222

# Logistic Regression

In [73]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [74]:
y_pred = logreg.predict(X_test)
accuracy_score(y_test,y_pred)

0.7942222222222223

In [76]:
logreg.coef_

array([[-3.95303727,  0.27987119, -0.27674464,  0.00437173,  0.28435408,
        -1.67140312, -0.54317254, -0.12512815, -0.48540144,  0.1141115 ,
         0.34939254, -0.57308927,  0.03503042, -0.14283186,  0.04526449,
         0.17095869,  0.24120671, -1.44363269,  0.84227135,  0.23087499]])

In [81]:
dict_coef = {'columns':list(X_train.columns),
             'coef':logreg.coef_[0]}

In [82]:
df_coef = pd.DataFrame(dict_coef)

In [84]:
df_coef.sort_values('coef',ascending=False)

,columns,coef
18,salary_low,0.842271
10,sales_hr,0.349393
4,time_spend_company,0.284354
1,last_evaluation,0.279871
16,sales_technical,0.241207
19,salary_medium,0.230875
15,sales_support,0.170959
9,sales_accounting,0.114112
14,sales_sales,0.045264
12,sales_marketing,0.035030
